<a href="https://colab.research.google.com/github/francoisdoanp/MLTBP/blob/master/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine learning - Final Project**

# Turbofan engine degradation dataset (NASA)

# Data Preparation

**Importing necessary packages**



In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import math

**Importing the Turbofan engine degradation dataset.**

**Files are located in the following Github repository: https://github.com/francoisdoanp/MLTBP**

We have 4 training datasets, which contains information about one hundred engines, all of the same type. Thus, we will combine the training and test data sets. 

The training and test sets have 21 columns: ID, Time (Cycles), 3 columns for operational settings and 21 sensor measurements.

The training and testing sets have the same format, while the validation sets only contain the real RUL (remaining useful life).

For more information on the data, consult the read me at the following address:https://github.com/francoisdoanp/MLTBP/blob/master/readme.txt

In [0]:
url_base = 'https://raw.githubusercontent.com/francoisdoanp/MLTBP/master/'

file_train_1 = 'train_FD001.txt'
file_train_2 = 'train_FD002.txt'
file_train_3 = 'train_FD003.txt'
file_train_4 = 'train_FD004.txt'

file_test_1 = 'test_FD001.txt'
file_test_2 = 'test_FD002.txt'
file_test_3 = 'test_FD003.txt'
file_test_4 = 'test_FD004.txt'

file_valid_1 = 'RUL_FD001.txt'
file_valid_2 = 'RUL_FD002.txt'
file_valid_3 = 'RUL_FD003.txt'
file_valid_4 = 'RUL_FD004.txt'


pt1 = pd.read_csv(url_base + file_train_1, sep=' ', header=None)
pt2 = pd.read_csv(url_base + file_train_2, sep=' ', header=None)
pt3 = pd.read_csv(url_base + file_train_3, sep=' ', header=None)
pt4 = pd.read_csv(url_base + file_train_4, sep=' ', header=None)

pte1 = pd.read_csv(url_base + file_test_1, sep=' ', header=None)
pte2 = pd.read_csv(url_base + file_test_2, sep=' ', header=None)
pte3 = pd.read_csv(url_base + file_test_3, sep=' ', header=None)
pte4 = pd.read_csv(url_base + file_test_4, sep=' ', header=None)

pv1 = pd.read_csv(url_base + file_valid_1, header=None)
pv2 = pd.read_csv(url_base + file_valid_2, header=None)
pv3 = pd.read_csv(url_base + file_valid_3, header=None)
pv4 = pd.read_csv(url_base + file_valid_4, header=None)


# Updating unit numbers

pt2[0] = pt2[0].apply(lambda x: x+100)
pt3[0] = pt3[0].apply(lambda x: x+360)
pt4[0] = pt4[0].apply(lambda x: x+460)

pte2[0] = pte2[0].apply(lambda x: x+100)
pte3[0] = pte3[0].apply(lambda x: x+359)
pte4[0] = pte4[0].apply(lambda x: x+459)


# Joining the dataframes

train_pd = pd.concat([pt1,pt2,pt3,pt4])
test_pd = pd.concat([pte1,pte2,pte3,pte4])
valid_pd = pd.concat([pv1,pv2,pv3,pv4], ignore_index=True)

train_pd = train_pd.drop(train_pd.columns[[26,27]], axis='columns')
test_pd = test_pd.drop(test_pd.columns[[26,27]], axis='columns')


# Assigning labels to Dataframe's columns based on the Readme

train_pd.columns = ['Unit Number', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
test_pd.columns = ['Unit Number', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
valid_pd.columns = ['RUL']

#Loading scaler

scaler = StandardScaler()
  

**Adding variables Conditons and fault mode**

Note:

**Condition (ONE)** and **Fault ONE** are binary variables.

When Condition(ONE) = 1 (true), it means that the condition is at Sea Level

When Condition(ONE) = 0 (false), it means NO, the condition IS NOT AT SEA LEVEL, and thus is the  second condition; SIX.

When Fault ONE = 1 (true), it means that the fault modes is one (HPC Degradation)

When Fault ONE = 0 (false), it means that the fault mode is TWO (HPC Degradation and Fan degradation)

In [0]:
# Adding variables Condition and fault modes

def value_condition_train(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 360) & (row['Unit Number'] > 100):
    return 0
  elif (row['Unit Number'] <= 460) & (row['Unit Number'] > 360):
    return 1
  else:
    return 0
  
def value_fault_train(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 360) & (row['Unit Number'] > 100):
    return 1
  elif (row['Unit Number'] <= 460) & (row['Unit Number'] > 360):
    return 0
  else:
    return 0
  
def value_condition_test(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 359) & (row['Unit Number'] > 100):
    return 0
  elif (row['Unit Number'] <= 459) & (row['Unit Number'] > 359):
    return 1
  else:
    return 0
  
def value_fault_test(row):
  if (row['Unit Number'] <= 100):
    return 1
  elif (row['Unit Number'] <= 359) & (row['Unit Number'] > 100):
    return 1
  elif (row['Unit Number'] <= 459) & (row['Unit Number'] > 359):
    return 0
  else:
    return 0


train_pd['Condition (One)'] = train_pd.apply(value_condition_train, axis=1)
train_pd['Fault ONE'] = train_pd.apply(value_fault_train,axis=1)

test_pd['Condition (One)'] = test_pd.apply(value_condition_test, axis=1)
test_pd['Fault ONE'] = test_pd.apply(value_fault_test,axis=1)

At this stage, we create the truth remaining useful life (RUL) for the training set.

Important note: In the training set, the last Cycle (represented in the table by 'Time (Cycles)') is when the engine is considered unusable. However, in the test set, the last cycle IS NOT when the engine is considered unusable. It will fail at a later time. Thus, in the valid_pd, we have the true RUL. 

In [0]:
#Adding column for remaining useful life (RUL)

y_train = pd.DataFrame(train_pd.groupby(['Unit Number'])['Time (Cycles)'].max())

train_pd = pd.merge(train_pd,y_train, on='Unit Number')
train_pd['RUL'] = train_pd['Time (Cycles)_y'] - train_pd['Time (Cycles)_x']
train_pd = train_pd.drop('Time (Cycles)_y',1)
train_pd = train_pd.rename(columns = {'Time (Cycles)_x':'Time (Cycles)'})

y_train = train_pd.iloc[:,28]


In [0]:
def adjusted_error(y_true, y_pred):
  pred_df = pd.DataFrame(y_pred)
  test_err = pd.concat([y_true,pred_df], axis=1)
  test_err.columns = ['RUL', 'Pred_RUL']
  a1 = 10
  a2 = 13
  score=0

  for index, row in test_err.iterrows():
    d = row['Pred_RUL'] - row['RUL']
    if d < 0:
      score += np.expm1(-(d/a1))
    else:
      score += np.expm1(d/a2)

  return score


# Data Exploration

**Now that we have our data, we can get to know our dataset**

In [0]:
display(train_pd.head())

display(test_pd)


print(f'The training dataset contains {train_pd.shape[0]} rows.')


In [0]:
# Print out a table with count, mean, std, min, max, 25%, 50%, 75%

print(train_pd.describe())

**Exploring the effects of sensors on RUL**



In [0]:


print(sns.heatmap(train_pd.corr(), annot=True, cmap='RdYlGn'))

train_pd.corr()

In [0]:
print()

**Exploring correlation between variables**

In [0]:
corr = train_pd.groupby(['RUL']).corr()
corr.style.background_gradient(cmap='coolwarm')

**How do we know when an engine fails?**

We look at the last time cycle for every unit number.



In [93]:
train_pd.groupby(['Unit Number'])['Time (Cycles)'].max().hist(bins=30, grid=False)
plt.xlabel('Number of Cycles')
plt.ylabel('Frequency')


NameError: ignored

#**Model 1: Multiple Linear Regression (Not time-sensitive)**






**Preparing data**

In [0]:
# Scaling data

train_pd_scaled = train_pd.copy()
train_pd_scaled.iloc[:,2:26] = scaler.fit_transform(train_pd.iloc[:,2:26])

test_pd_lm = test_pd.copy()
test_pd_lm.iloc[:,2:26] = scaler.fit_transform(test_pd.iloc[:,2:26])

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)



In [0]:
# Removing RUL and Unit Number columns, as we do not want these features to be in our predictors

train_pd_lm = train_pd_scaled.copy()
train_pd_lm = train_pd_lm.drop(['RUL', 'Unit Number'],axis=1)

# Keeping only last time cycle for each unit number

idx = test_pd_lm.groupby(['Unit Number'])['Time (Cycles)'].transform(max) == test_pd_lm['Time (Cycles)']
test_pd_lm = test_pd_scaled[idx]

# Removing Unit Number column

test_pd_lm = test_pd_lm.drop(['Unit Number'], axis=1)

**Fitting Linear Model**

In [103]:
# Fitting Linear model

reg = LinearRegression().fit(train_pd_lm, y_train)

y_pred = reg.predict(test_pd_lm)

acc_score_lm =  metrics.mean_squared_error(valid_pd, y_pred)

print(f'The square root of the mean squared error for the linear model is: {math.sqrt(acc_score_lm)}.')


The square root of the mean squared error for the linear model is: 54.8991731927702.


In [0]:
print(adjusted_error(valid_pd, y_pred))